## Imports and Setup

In [78]:
## Project Imports
import gspread
import os
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [79]:
pres_data = pd.read_excel(r"C:\Users\gr0581vr\Desktop\homework\2022-2024-elections-main\full-data.xlsx", sheet_name=0)
sen_data = pd.read_excel(r"C:\Users\gr0581vr\Desktop\homework\2022-2024-elections-main\full-data.xlsx", sheet_name=1)

## A Bit of Data Exploration

Mostly just a view into different features as part of the models (will elaborate this later in the final paper and within this notebook).

In [80]:
pres_data.head()

,State,Region,2010 Population,2020 Population,pres2000,pres2004,pres2008,pres2012,sen1-3rdrecent,sen2-3rdrecent,...,dem-finance,median-hh-income,recent-five-polling-avg-16,polling-party-lead-16,recent-five-polling-avg-20,polling-party-lead-20,pvi,party-pvi,pres2016,pres2020
0,Alabama,South,4779736,5024279,0,0,0,0,0,0,...,37491.77,52035,16,0,19.5,0,15,0,0,0
1,Alaska,West,710231,733391,0,0,0,0,1,0,...,3566.69,77790,4,1,7.7,0,9,0,0,0
2,Arizona,West,6392017,7151502,0,0,0,0,0,0,...,38865284.52,61529,2,0,2.6,1,3,0,0,1
3,Arkansas,South,2915918,3011524,0,0,0,0,1,1,...,26852.08,49475,23,0,22.8,0,16,0,0,0
4,California,West,37253956,39538223,1,1,1,1,1,1,...,9467165.58,78672,28,1,29.2,1,14,1,1,1


In [81]:
sen_data.head()

,State,Region,2020 Population,pres2000,pres2004,pres2008,pres2012,pres2016,pres2020,sen1-recent,...,rep-finance,dem-finance,median-hh-income,recent-five-polling-avg,polling-party-lead,pvi,party-pvi,employment-rate,urban-pct-2010,recent-res
0,Alabama,South,5024279,0,0,0,0,0,0,0,...,6829526.55,37491.77,52035,20.4,0,15,0,54.0,59.04,0.0
1,Alaska,West,733391,0,0,0,0,0,0,0,...,7527972.46,3566.69,77790,6.5,0,9,0,59.6,66.02,NaN
2,Arizona,West,7151502,0,0,0,0,0,1,1,...,13839094.88,38865284.52,61529,7.0,1,3,0,56.2,89.81,NaN
3,Arkansas,South,3011524,0,0,0,0,0,0,0,...,5973540.80,26852.08,49475,45.0,0,16,0,54.8,56.16,0.0
4,California,West,39538223,1,1,1,1,1,1,1,...,135852.28,9467165.58,78672,25.0,1,14,1,59.4,94.95,1.0


In [82]:
## First we form predictions for presidential data
## We will use 2016 results as a training set, this will involve using all 
## features above except for 2020 population and 2020 polling info columns 
## (and obviously the )


le = LabelEncoder()
region_enc = le.fit_transform(pres_data['Region'])
pres_data["Region"] = region_enc
sen_data["Region"] = region_enc

In [83]:
noninc_cols_train = ['State','2020 Population','recent-five-polling-avg-20','polling-party-lead-20','pres2016','pres2020']
pres_train_X = pres_data.drop(noninc_cols_train, axis=1)
pres_train_y = pres_data[['pres2016']]

pres_train_X.to_csv(cwd + "\\pres_train_X.csv")
pres_train_y.to_csv(cwd + "\\pres_train_y.csv")

In [84]:
pres_train_X.head()

,Region,2010 Population,pres2000,pres2004,pres2008,pres2012,sen1-3rdrecent,sen2-3rdrecent,gdp,high-school-pop,...,white,inc-party,inc-w-recent,rep-finance,dem-finance,median-hh-income,recent-five-polling-avg-16,polling-party-lead-16,pvi,party-pvi
0,2,4779736,0,0,0,0,0,0,226896.5,30.3,...,0.640074,1,1,6829526.55,37491.77,52035,16,0,15,0
1,3,710231,0,0,0,0,1,0,49820.0,28.4,...,0.627645,0,1,7527972.46,3566.69,77790,4,1,9,0
2,3,6392017,0,0,0,0,0,0,373719.0,23.8,...,0.517834,0,0,13839094.88,38865284.52,61529,2,0,3,0
3,2,2915918,0,0,0,0,1,1,130750.7,33.9,...,0.692194,1,1,5973540.80,26852.08,49475,23,0,16,0
4,3,37253956,1,1,1,1,1,1,3007187.7,20.4,...,0.330164,1,1,135852.28,9467165.58,78672,28,1,14,1


In [85]:
pres_train_y.head()

,pres2016
0,0
1,0
2,0
3,0
4,1


## Model Creation

Models here are: logistic regression with no penalty, logistic regression models with l1 and l2 penalization terms, random forests, xgboost.

Note for logistic regression we employ a standard scaling as necessary when applying regularization terms. We keep the standardization even for no-penalty (regular) logistic regression just for the sake of completion. No standardization is done for 

In [86]:
## Logistic Regression with Scaling
scaler = StandardScaler()
lr = LogisticRegression(penalty='none', solver = 'saga')
lrmodel = Pipeline([('standardize', scaler), ('log_reg', lr)])
lrmodel.fit(pres_train_X,pres_train_y.values.ravel())

## L1-Penalty Logistic Regression ("LASSO-like" logistic regression)
lassor = LogisticRegression(penalty='l1', solver = 'saga')
lassomodel = Pipeline([('standardize', scaler), ('lasso_log_reg', lassor)])
lassomodel.fit(pres_train_X,pres_train_y.values.ravel())

## L2-Penalty Logistic Regression
ridger = LogisticRegression()
ridgemodel = Pipeline([('standardize', scaler), ('ridge_log_reg', ridger)])
ridgemodel.fit(pres_train_X,pres_train_y.values.ravel())

C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Pipeline(steps=[('standardize', StandardScaler()),
                ('ridge_log_reg', LogisticRegression())])

In [87]:
## Decision Trees
dtree = DecisionTreeClassifier(random_state = 83)
dtree.fit(pres_train_X,pres_train_y.values.ravel())

DecisionTreeClassifier(random_state=83)

In [89]:
## Random Forests
forest = RandomForestClassifier(n_estimators = 120, random_state = 83)
forest.fit(pres_train_X,pres_train_y.values.ravel())

RandomForestClassifier(n_estimators=120, random_state=83)

In [88]:
## XGBoost
xgboost1 = xgb.XGBClassifier(base_score=0.5, booster='gbtree', random_state = 83, colsample_bytree = 0.3, learning_rate = 0.1, alpha = 10, n_estimators = 120)
xgboost1.fit(pres_train_X,pres_train_y.values.ravel())

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=120, n_jobs=0,
              num_parallel_tree=1, predictor='auto', ...)

## Some Inference Info (Coefficients)

In [90]:
## Coefficients organized from least important to most important
print(np.sort(lr.coef_))
print(np.argsort(np.array(lr.coef_)))

print(np.sort(lassor.coef_))
print(np.argsort(np.array(lassor.coef_)))

print(np.sort(ridger.coef_))
print(np.argsort(np.array(ridger.coef_)))

[[-0.62924399 -0.55215047 -0.51902698 -0.41083646 -0.33288247 -0.32598983
  -0.29653656 -0.15251823 -0.1338335  -0.11631535  0.04102852  0.10162632
   0.13348677  0.16002361  0.21672083  0.22755982  0.24411753  0.27098843
   0.3312234   0.35850606  0.38032833  0.45160797  0.4768807   0.47796315
   0.4882422   0.49489524  0.50550432  0.56191194  0.71938719  0.76554884
   1.9082482 ]]
[[ 9 24  1 11 21  6  8 15 17 10 23  0 22 29  2  3 18 12 19 20 27  4  5  7
  16 25 26 14 13 28 30]]
[[-0.01814146  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.12727483  0.20210814  0.26590525  0.65020297
   2.43745913]]
[[ 9  0 27 26 25 24 23 22 21 20 19 18 17 16 29 15 12 11 10  8  6  4  3  2
   1 14  7  5 13 28 30]]
[[-0.40214395 -0.37667092 -0.28359664 -0.21455842 -0.15021314 

In [91]:
## Get Tree Representation
text_representation = tree.export_text(dtree)
print(text_representation)

|--- feature_30 <= 0.50
|   |--- class: 0
|--- feature_30 >  0.50
|   |--- class: 1



In [92]:
## Feature Importances for Ensembles
print(np.sort(dtree.feature_importances_))
print(np.argsort(np.array(dtree.feature_importances_)))
print(np.sort(forest.feature_importances_))
print(np.argsort(np.array(forest.feature_importances_)))
print(np.sort(xgboost1.feature_importances_))
print(np.argsort(np.array(xgboost1.feature_importances_)))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1.]
[ 0 28 27 26 25 24 23 22 21 20 19 18 17 16 29 15 13 12 11 10  9  8  7  6
  5  4  3  2  1 14 30]
[0.         0.00136609 0.0027355  0.00519779 0.00554901 0.00579912
 0.00656748 0.00728432 0.00760413 0.00809159 0.00954432 0.00963875
 0.01029661 0.01032012 0.01124802 0.01372405 0.01460365 0.02029505
 0.03013778 0.03021072 0.0319415  0.03309494 0.03346494 0.041301
 0.04318408 0.04996164 0.05742606 0.06927306 0.08075092 0.14893616
 0.20045158]
[ 6 23 15 17 11 19  8  1 27 25 14 22 29  0 18 10 21 20  9 24  4  3 12 16
  7  2  5 13 26 28 30]
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.06907645 0.08975542 0.12933445 0.27262205
 0.43921164]
[ 0 27 25 24 23 22 21 20 19 18 16 29 14 15 12  1  

In [93]:
print(pres_train_X.columns[30])
print(pres_train_X.columns[28])
print(pres_train_X.columns[7])
print(pres_train_X.columns[5])

print(pres_train_X.columns[26])
print(pres_train_X.columns[2])
print(pres_train_X.columns[13])

party-pvi
polling-party-lead-16
sen2-3rdrecent
pres2012
median-hh-income
pres2000
grad-professional


## Creating a Test Set

Test set here is similar features to the training data, however we make the following changes:

*   Obviously remove any 2016 related info, so we remove 2010 population counts and the 2016 presidential polling averages.
*   From the training data, we fit the model with using 2000 presidential election results, but instead we will use 



In [94]:
noninc_cols_test = ['State','2010 Population','recent-five-polling-avg-16','polling-party-lead-16','pres2000','pres2020']
pres_test_X = pres_data.drop(noninc_cols_test, axis=1)
pres_test_y = pres_data[['pres2020']]

## Move 2016 results to position where 2012 results
## Here we use 2004, 2008, 2012, 2016 results as predictors towards 2020 results
## Since 2000 was removed we have to rearrange columns here
col_move = pres_test_X.pop("pres2016")
pres_test_X.insert(2, "pres2016", col_move)

## Rename columns just so the models can run
## A little bit hacky but we turn the 2000 results into 2016 results
dict = {'2020 Population': '2010 Population',
        'polling-party-lead-20': 'polling-party-lead-16',
        'pres2016': 'pres2000',
        'recent-five-polling-avg-20': 'recent-five-polling-avg-16',}
 
# call rename () method
pres_test_X.rename(columns=dict,inplace=True)

pres_test_X.to_csv(cwd + "\\pres_test_X.csv")
pres_test_y.to_csv(cwd + "\\pres_test_y.csv")

In [95]:
pres_test_X.head() ## We see the data matches the format as our train set :)

,Region,2010 Population,pres2000,pres2004,pres2008,pres2012,sen1-3rdrecent,sen2-3rdrecent,gdp,high-school-pop,...,white,inc-party,inc-w-recent,rep-finance,dem-finance,median-hh-income,recent-five-polling-avg-16,polling-party-lead-16,pvi,party-pvi
0,2,5024279,0,0,0,0,0,0,226896.5,30.3,...,0.640074,1,1,6829526.55,37491.77,52035,19.5,0,15,0
1,3,733391,0,0,0,0,1,0,49820.0,28.4,...,0.627645,0,1,7527972.46,3566.69,77790,7.7,0,9,0
2,3,7151502,0,0,0,0,0,0,373719.0,23.8,...,0.517834,0,0,13839094.88,38865284.52,61529,2.6,1,3,0
3,2,3011524,0,0,0,0,1,1,130750.7,33.9,...,0.692194,1,1,5973540.80,26852.08,49475,22.8,0,16,0
4,3,39538223,1,1,1,1,1,1,3007187.7,20.4,...,0.330164,1,1,135852.28,9467165.58,78672,29.2,1,14,1


In [96]:
pres_test_y.head()　

SyntaxError: invalid non-printable character U+3000 (226488710.py, line 1)

In [97]:
## Form predictions over all models
logpreds = lrmodel.predict(pres_test_X)
lassopreds = lassomodel.predict(pres_test_X)
ridgepreds = ridgemodel.predict(pres_test_X)
treepreds = dtree.predict(pres_test_X)
rfpreds = forest.predict(pres_test_X)
xgbpreds = xgboost1.predict(pres_test_X)

In [98]:
## Notably, all the classification results are the same
print(logpreds)
print(lassopreds)
print(ridgepreds)
print(treepreds)
print(rfpreds)
print(xgbpreds)

[0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]
[0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]
[0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]
[0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]
[0 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]
[0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 1
 0 1 0 0 0 0 0 1 1 1 0 0 0]


In [99]:
preds = np.array(xgbpreds)
true = np.array(pres_test_y.values.ravel())
print(np.equal(preds,true))
print(np.where(np.not_equal(preds,true)))

[ True  True False  True  True  True  True  True  True False False  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True  True False  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
 False  True]
(array([ 2,  9, 10, 21, 27, 37, 48], dtype=int64),)


From the above difference checking between predictions and true values, we see that indices 2,9,21,37,48 in our states data were misclassified. These correspond in the data to the following states: Arizona, Georgia, Michigan, Pennsylvania, and Wisconsin. These were all flips between the 2016 and 2020 election.

## Senate Predictions

Here we form predictions for the US senate for the upcoming 2022 Midterm Elections.

As a note, the preprocessing here for the senate data is altered. We have a set of 13 "competitive" states that are considered competitive based off [2022 US Senate Election Ratings](https://en.wikipedia.org/wiki/2022_United_States_Senate_elections#Predictions). These will form our test set,and the training set will be over other states where (1) they have no Senate competition in 2022 so we use their most recent party outcome for the most recent Senate election in 2020 or (2) forecasting and demographic/polling factors strongly indicate a party direction for the Senate results, to the point where no forecasting is necessary and polling is usually sparse. The paper addresses this in more depth as a potential challenge.

In [100]:
null_results = pd.isnull(sen_data["recent-res"])
competitive_states = [i for i, n in enumerate(null_results) if n == True] 
other_states = list(set(list(range(50))) - set(competitive_states))
print(other_states)
print(competitive_states)

sen_data = sen_data.drop(["State"], axis = 1)
sen_train_X = sen_data.iloc[other_states]
sen_train_X = sen_train_X.drop(["recent-res"], axis = 1)
sen_train_y = sen_data[~sen_data['recent-res'].isnull()]['recent-res']
sen_test_X = sen_data.iloc[competitive_states]
sen_test_X = sen_test_X.drop(["recent-res"], axis = 1)

## Write CSVs, this is for inference analysis in R for logistic models
sen_train_X.to_csv(cwd + "\\sen_train_X.csv")
sen_train_y.to_csv(cwd + "\\sen_train_y.csv")
sen_test_X.to_csv(cwd + "\\sen_test_X.csv")

[0, 3, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 29, 30, 31, 33, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 47, 49]
[1, 2, 5, 8, 9, 24, 27, 28, 32, 34, 37, 46, 48]


In [101]:
sen_train_X.head()

,Region,2020 Population,pres2000,pres2004,pres2008,pres2012,pres2016,pres2020,sen1-recent,sen2-recent,...,inc-w-recent,rep-finance,dem-finance,median-hh-income,recent-five-polling-avg,polling-party-lead,pvi,party-pvi,employment-rate,urban-pct-2010
0,2,5024279,0,0,0,0,0,0,0,0,...,1,6829526.55,37491.77,52035,20.4,0,15,0,54.0,59.04
3,2,3011524,0,0,0,0,0,0,0,0,...,1,5973540.80,26852.08,49475,45.0,0,16,0,54.8,56.16
4,3,39538223,1,1,1,1,1,1,1,1,...,1,135852.28,9467165.58,78672,25.0,1,14,1,59.4,94.95
6,0,3605944,1,1,1,1,1,1,1,1,...,1,1088131.76,7442253.87,79855,19.6,1,7,1,61.7,87.99
7,2,989948,1,1,1,1,1,1,1,1,...,1,479515.11,7217773.49,69110,30.0,1,6,1,58.0,83.30


In [102]:
sen_train_y.head()

0    0.0
3    0.0
4    1.0
6    1.0
7    1.0
Name: recent-res, dtype: float64

In [103]:
sen_test_X.head()

,Region,2020 Population,pres2000,pres2004,pres2008,pres2012,pres2016,pres2020,sen1-recent,sen2-recent,...,inc-w-recent,rep-finance,dem-finance,median-hh-income,recent-five-polling-avg,polling-party-lead,pvi,party-pvi,employment-rate,urban-pct-2010
1,3,733391,0,0,0,0,0,0,0,0,...,1,7527972.46,3566.69,77790,6.5,0,9,0,59.6,66.02
2,3,7151502,0,0,0,0,0,1,1,1,...,0,13839094.88,38865284.52,61529,7.0,1,3,0,56.2,89.81
5,3,5773714,0,0,1,1,1,1,1,1,...,0,0.00,11162092.65,75231,8.5,1,3,1,64.3,86.15
8,2,21538187,0,0,1,1,0,0,0,0,...,0,30159319.08,30773246.01,57703,9.0,0,3,0,55.4,91.16
9,2,10711908,0,0,0,0,0,1,1,1,...,0,16470998.24,73223096.79,61224,0.5,0,3,0,59.2,75.07


In [104]:
lrmodel.fit(sen_train_X,sen_train_y.values.ravel())
lassomodel.fit(sen_train_X,sen_train_y.values.ravel())
ridgemodel.fit(sen_train_X,sen_train_y.values.ravel())
dtree.fit(sen_train_X,sen_train_y.values.ravel())
forest.fit(sen_train_X,sen_train_y.values.ravel())
xgboost1.fit(sen_train_X,sen_train_y.values.ravel())

C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\gr0581vr\Anaconda3\envs\homework\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=120, n_jobs=0,
              num_parallel_tree=1, predictor='auto', ...)

In [105]:
## Form predictions over all models
logpreds = lrmodel.predict(sen_test_X)
lassopreds = lassomodel.predict(sen_test_X)
ridgepreds = ridgemodel.predict(sen_test_X)
treepreds = dtree.predict(sen_test_X)
rfpreds = forest.predict(sen_test_X)
xgbpreds = xgboost1.predict(sen_test_X)

In [106]:
## Differences here!
print(logpreds)
print(lassopreds)
print(ridgepreds)
print(treepreds)
print(rfpreds)
print(xgbpreds)

[0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1.]
[0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1.]
[0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 0]


The indices above correspond to the following states: [Alaska, Arizona, Colorado, Florida, Georgia, Missouri, Nevada, New Hampshire, North Carolina, Ohio, Pennsylvania, Washington, Wisconsin]

## More Inference for Senate Model

In [107]:
## Coefficients organized from least important to most important
print(np.sort(lr.coef_))
print(np.argsort(np.array(lr.coef_)))

print(np.sort(lassor.coef_))
print(np.argsort(np.array(lassor.coef_)))

print(np.sort(ridger.coef_))
print(np.argsort(np.array(ridger.coef_)))

[[-0.344335   -0.28817789 -0.27688123 -0.16632284 -0.10598584 -0.1050376
  -0.08717437 -0.06235964 -0.05241035 -0.00602263  0.00893146  0.05773487
   0.09565646  0.10926133  0.11212298  0.12259831  0.16082897  0.19165241
   0.1977994   0.23104798  0.23388761  0.26079025  0.26079025  0.27561306
   0.29714889  0.36170844  0.37538787  0.38381177  0.40167936  0.40500525
   0.42878154  0.48156581  0.48156581  0.48690937  0.49803335  0.5228862
   0.56563238  0.74696603  0.77140756]]
[[27 15 31 17 30 35 37 16 20 21  0 29 25 14  1 18 24 23 26 33 22  6 36 10
  32 19  4  2  8  3  5  7 34 12 38 28 13 11  9]]
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.02299569 0.04589954 0.04589954 0.06459875
  0.07700384 0.17379156 0.18806353 0.19299763 0.23595213 0.35857802
  0.35857802 0.7713

In [108]:
print(sen_train_X.columns[9])
print(sen_train_X.columns[11])
print(sen_train_X.columns[13])
print(sen_train_X.columns[34])
print(sen_train_X.columns[28])
print(sen_train_X.columns[7])
print(sen_train_X.columns[16])
print(sen_train_X.columns[15])
print(sen_train_X.columns[19])
print(sen_train_X.columns[38])

sen2-recent
sen2-2ndrecent
sen2-3rdrecent
polling-party-lead
inc-party
pres2020
some-college
high-school-pop
grad-professional
urban-pct-2010


In [109]:
## Get Tree Representation
text_representation = tree.export_text(dtree)
print(text_representation)

|--- feature_9 <= 0.50
|   |--- class: 0.0
|--- feature_9 >  0.50
|   |--- class: 1.0



In [110]:
## Feature Importances for Ensembles
print(np.sort(dtree.feature_importances_))
print(np.argsort(np.array(dtree.feature_importances_)))
print(np.sort(forest.feature_importances_))
print(np.argsort(np.array(forest.feature_importances_)))
print(np.sort(xgboost1.feature_importances_))
print(np.argsort(np.array(xgboost1.feature_importances_)))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[ 0 21 22 23 24 25 26 27 20 28 30 31 32 33 34 35 36 29 37 19 17  1  2  3
  4  5  6  7 18  8 10 11 12 13 14 15 16 38  9]
[0.         0.         0.         0.         0.         0.00025305
 0.00068519 0.00077083 0.00088916 0.00091766 0.00123467 0.00163139
 0.00180508 0.00231108 0.002372   0.00280412 0.00404633 0.00504132
 0.00644731 0.00691513 0.01043537 0.0111667  0.01175279 0.01782676
 0.01998911 0.02058077 0.02112438 0.02780488 0.03595817 0.03722988
 0.04234075 0.04881164 0.05141508 0.06861354 0.07005088 0.07988107
 0.0875037  0.11174531 0.18764492]
[24 21 18 26 29 16  0  1 35 25 23 27 37 14 12 17 30 33 15 10  3 20 31 13
  4 38 28 32  8  2 19 22 36  6  5  7 34 11  9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[ 0 21 22 23 24 25 26 27 20 28 30 31 32 33 34 35 36 29 37 19 17  1  2  3
  4  5  6 